In [1]:
import math
import os

In [2]:
def generate_p4(int_bits, frac_bits, num_newton_iterations):
    total_bits = int_bits + frac_bits
    FIXED_ONE = 1 << frac_bits
    FIXED_TWO = 2 << frac_bits

    normalize_if_blocks = ""
    for _ in range(total_bits):
        normalize_if_blocks += "        if (meta.norm_val >= FIXED_TWO) { meta.norm_val = meta.norm_val >> 1; meta.shift_amount = meta.shift_amount + 1; }\n"
    for _ in range(total_bits):
        normalize_if_blocks += "        if (meta.norm_val < FIXED_ONE) { meta.norm_val = meta.norm_val << 1; meta.shift_amount = meta.shift_amount - 1; }\n"

    newton_block = ""
    for i in range(num_newton_iterations):
        newton_block += "        fixed_mul(meta.norm_val, meta.temp_B);\n"
        newton_block += "        fixed_mul(meta.temp_B, (2 * FIXED_ONE - meta.temp_A));\n"
        if i < num_newton_iterations - 1:
            newton_block += "        meta.temp_B = meta.temp_A;\n"
    division_shift_blocks = ""
    for _ in range(total_bits):
        division_shift_blocks += "        if (meta.shift_amount > 0) {meta.temp_A = meta.temp_A >> 1; meta.shift_amount = meta.shift_amount - 1;} else if (meta.shift_amount < 0) {meta.temp_A = meta.temp_A << 1; meta.shift_amount = meta.shift_amount + 1;}\n"

    p4_code = f"""
#include <core.p4>
#include <v1model.p4>

typedef int<{total_bits}> fixed;
typedef bit<{total_bits}> number_req_t;
typedef bit<48> macAddr_t;

const bit<16> TYPE_CALC = 0xABAB;
const fixed FIXED_ONE = {FIXED_ONE};
const fixed FIXED_TWO = {FIXED_TWO};

header ethernet_t {{
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   ether_type;
}}

header calc_t {{
    bit<8> type;
    number_req_t a;
    number_req_t b;
}}

struct header_t {{
    ethernet_t ethernet;
    calc_t calc;
}}

struct metadata_t {{
    int<{math.ceil(math.log2(total_bits))}> shift_amount;
    fixed norm_val;
    fixed temp_A;
    fixed temp_B;
}}

parser SwitchParser(packet_in packet,
                    out header_t hdr,
                    inout metadata_t meta,
                    inout standard_metadata_t standard_metadata) {{
    state start {{
        transition parse_ethernet;
    }}

    state parse_ethernet {{
        packet.extract(hdr.ethernet);
        transition select(hdr.ethernet.ether_type) {{
            TYPE_CALC: parse_calc;
            default: accept;
        }}
    }}

    state parse_calc {{
        packet.extract(hdr.calc);
        transition accept;
    }}
}}

control SwitchVerifyChecksum(inout header_t hdr, inout metadata_t meta) {{
    apply {{}}
}}

control SwitchIngress(inout header_t hdr,
                      inout metadata_t meta,
                      inout standard_metadata_t standard_metadata) {{

    action fixed_mul(fixed a, fixed b){{
        meta.temp_A = (fixed)(((int<{total_bits*2}>)a * (int<{total_bits*2}>)b) >> {frac_bits});
    }}

    action normalize(fixed b) {{
        meta.norm_val = b;
        meta.shift_amount = 0;
{normalize_if_blocks}
    }}

    action fixed_division(fixed numerator, fixed denominator){{
        normalize(denominator);
        meta.temp_B = (FIXED_ONE + (FIXED_ONE >> 1)) - (meta.norm_val >> 1);
{newton_block}
{division_shift_blocks}
        fixed_mul(numerator, meta.temp_A);
    }}

    apply {{
        if (hdr.calc.type == 0 && standard_metadata.ingress_port == 0) {{
            fixed_division((int<{total_bits}>)hdr.calc.a, (int<{total_bits}>)hdr.calc.b);
            // Logging is commented for testing performance
            // log_msg("Division Result: {{}}", {{meta.temp_A}});
            hdr.calc.type = 1;
            hdr.calc.a = (bit<{total_bits}>)meta.temp_A;
            macAddr_t tmp_mac;
            tmp_mac = hdr.ethernet.srcAddr;
            hdr.ethernet.srcAddr = hdr.ethernet.dstAddr;
            hdr.ethernet.dstAddr = tmp_mac;
            standard_metadata.egress_spec = 1;
        }}
    }}
}}

control SwitchEgress(inout header_t hdr,
                     inout metadata_t meta,
                     inout standard_metadata_t standard_metadata) {{
    apply {{}}
}}

control SwitchComputeChecksum(inout header_t hdr, inout metadata_t meta) {{
    apply {{}}
}}

control SwitchDeparser(packet_out pkt,
                       in header_t hdr) {{
    apply {{
        pkt.emit(hdr);
    }}
}}

V1Switch(SwitchParser(),
         SwitchVerifyChecksum(),
         SwitchIngress(),
         SwitchEgress(),
         SwitchComputeChecksum(),
         SwitchDeparser()) main;
"""
    return p4_code

In [3]:
# if __name__ == "__main__":
#     generated_code = generate_p4(16, 16, 5)

#     script_dir = os.path.dirname(os.path.abspath(__file__))
#     output_path = os.path.join(script_dir, "newton_method_no_LUT.p4")
#     with open(output_path, "w", encoding="utf-8") as f:
#         f.write(generated_code)

In [5]:
if __name__ == "__main__":
    generated_code = generate_p4(1, 1, 1)
    print(generated_code)


#include <core.p4>
#include <v1model.p4>

typedef int<2> fixed;
typedef bit<2> number_req_t;
typedef bit<48> macAddr_t;

const bit<16> TYPE_CALC = 0xABAB;
const fixed FIXED_ONE = 2;
const fixed FIXED_TWO = 4;

header ethernet_t {
    macAddr_t dstAddr;
    macAddr_t srcAddr;
    bit<16>   ether_type;
}

header calc_t {
    bit<8> type;
    number_req_t a;
    number_req_t b;
}

struct header_t {
    ethernet_t ethernet;
    calc_t calc;
}

struct metadata_t {
    int<1> shift_amount;
    fixed norm_val;
    fixed temp_A;
    fixed temp_B;
}

parser SwitchParser(packet_in packet,
                    out header_t hdr,
                    inout metadata_t meta,
                    inout standard_metadata_t standard_metadata) {
    state start {
        transition parse_ethernet;
    }

    state parse_ethernet {
        packet.extract(hdr.ethernet);
        transition select(hdr.ethernet.ether_type) {
            TYPE_CALC: parse_calc;
            default: accept;
        }
    }

    st